In [1]:
from os import path, mkdir, makedirs
from PIL import Image
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import concurrent.futures
import os
import threading
import random

In [2]:
output_folder = "output/a"
if not path.exists(output_folder):
    os.makedirs(output_folder)
output_folder = "output/b"
if not path.exists(output_folder):
    os.makedirs(output_folder)
output_folder = "output/c"
if not path.exists(output_folder):
    os.makedirs(output_folder)
output_folder = "output/d"
if not path.exists(output_folder):
    os.makedirs(output_folder)

backgrounds = ["countryside", "desert", "forest", "glacial","savannah","sandshore","beach","forest_night"]
characters = ["mage", "warrior", "pirate", "monster", "toadking"]
objects = ["none", "barrel", "anchor", "axe", "bomb", "key", "chest", "bananas", "cupcake", "donut", "heart",]

In [3]:
police_options = ['equipped_p','unarmed_p']
civilian_options = ['equipped_c','unarmed_c']

In [4]:
backgrounds_count = 0
equipped_p_count = 0
unarmed_p_count = 0
equipped_c_count = 0
unarmed_c_count = 0
for root, dirs, files in os.walk(".", topdown=False):
    for name in files:
        #print(os.path.join(root, name))
        if "backgrounds" in os.path.join(root, name):
            backgrounds_count += 1
        if "equipped_c" in os.path.join(root, name):
            equipped_c_count += 1
        if "unarmed_c" in os.path.join(root, name):
            unarmed_c_count += 1
        if "equipped_p" in os.path.join(root, name):
            equipped_p_count += 1
        if "unarmed_p" in os.path.join(root, name):
            unarmed_p_count += 1
terrorist_count = equipped_c_count + unarmed_c_count
police_count = equipped_p_count + unarmed_p_count

In [8]:
def generate_image(classes, background_file_name, police_file_name, civilian_file_name, file_name):
    """Generate image with given background, given character and given object and save it with the given file name

    Args:
        background (str): background name
        character (str): character name
        object (str): object name
        file_name (str): file name
    """
    background_file = path.join("backgrounds", background_file_name)
    background_image = Image.open(background_file)
    background_image = background_image.resize((1920,1080))

    #Create character
    police_character_file = path.join("characters/police_chars", police_file_name)
    police_character_image = Image.open(police_character_file)
    police_character_image = police_character_image.resize((int(police_character_image.size[0]/3),int(police_character_image.size[1]/3)))
    police_character_coordinates = (int(1920/2-police_character_image.width*1.6), int(1080-police_character_image.height*1.1)) #x, y
    civilian_character_file = path.join("characters/civilian_chars", civilian_file_name)
    civilian_character_image = Image.open(civilian_character_file)
    civilian_character_image = civilian_character_image.resize((int(civilian_character_image.size[0]/3),int(civilian_character_image.size[1]/3)))
    civilian_character_coordinates = (int(1920/2+civilian_character_image.width*1.6), int(1080-civilian_character_image.height)) #x, y
    background_image.paste(police_character_image, police_character_coordinates, mask=police_character_image)
    background_image.paste(civilian_character_image, civilian_character_coordinates, mask=civilian_character_image)
    background_image.show()
    output_file = path.join("output/"+classes, f"{file_name}")
    background_image.save(output_file)

def generate_all_imgs():
    """Generate all possible combination of images
    """
    num = 0
    df = pd.DataFrame(columns = ["background", "character", "object", "generated image"])
    for background in backgrounds:
        for character in characters:
            for object in objects:
                generate_image(background, character, object, f"generated{num}")
                data = [background, character, object, f"generated{num}"]
                s = pd.Series(data, index=df.columns)
                df = df.append(s, ignore_index=True)
                num += 1
    df.to_csv('data.csv', index=False)

def generate_random_imgs(total_imgs,num):
    """Generates a given number of random images according to predefined probabilities

    Args:
        total_imgs (int): total number of images to generate
    """
    df = pd.DataFrame(columns = ["image", "class", "civilian_present?", "civilian_armed?", "police_present?","police_armed?", "action"])
    for img in tqdm(range(num, num+total_imgs)):
        background_character_number = random.randint(0,backgrounds_count-1)
        background_file_name = "background" + str(background_character_number) + ".png"
        police_option = random.choice(police_options)
        police_character_number = random.randint(0,globals()['%s_count'% police_option]-1)
        police_file_name = police_option + "/" + police_option + "_" + str(police_character_number) + ".png"
        civilian_option = random.choice(civilian_options)
        #rint(civilian_option)
        civilian_character_number = random.randint(0,globals()['%s_count'% civilian_option]-1)
        civilian_file_name = civilian_option + "/" + civilian_option + "_" + str(civilian_character_number) + ".png"
        
        civilian_armed = 1 if civilian_option == "equipped_c" else 0
        police_armed = 1 if police_option == "equipped_p" else 0
        if civilian_armed==1 and police_armed==1:
            classes = "a"
        if civilian_armed==1 and police_armed==0:
            classes = "b" 
        if civilian_armed==0 and police_armed==1:
            classes = "c" 
        if civilian_armed==0 and police_armed==0:
            classes = "d"
        
        generate_image(classes, background_file_name, police_file_name, civilian_file_name, f"img{img}.png")
        
        data = [f"img{img}.png",classes, 1,civilian_armed,1,police_armed,"TBD"]
        s = pd.Series(data, index=df.columns)
        df = df.append(s, ignore_index=True)

    df.to_csv('data' + str(num)+ '.csv', index=False)

In [17]:
if __name__ == "__main__":
    #generate_all_imgs()
    t1 = threading.Thread(target=generate_random_imgs, args=(1000,0))
    t2 = threading.Thread(target=generate_random_imgs, args=(1000,1000))
    t3 = threading.Thread(target=generate_random_imgs, args=(1000,2000))
    t4 = threading.Thread(target=generate_random_imgs, args=(1000,3000))
    t5 = threading.Thread(target=generate_random_imgs, args=(1000,4000))
    t6 = threading.Thread(target=generate_random_imgs, args=(1000,5000))
    t7 = threading.Thread(target=generate_random_imgs, args=(1000,6000))
    t8 = threading.Thread(target=generate_random_imgs, args=(1000,7000))
    t9 = threading.Thread(target=generate_random_imgs, args=(1000,8000))
    t10 = threading.Thread(target=generate_random_imgs, args=(1000,9000))
    t11 = threading.Thread(target=generate_random_imgs, args=(1000,10000))
    t12 = threading.Thread(target=generate_random_imgs, args=(1000,11000))
    t13 = threading.Thread(target=generate_random_imgs, args=(1000,12000))
    t14 = threading.Thread(target=generate_random_imgs, args=(1000,13000))
    t15 = threading.Thread(target=generate_random_imgs, args=(1000,14000))
    t16 = threading.Thread(target=generate_random_imgs, args=(1000,15000))
    t17 = threading.Thread(target=generate_random_imgs, args=(1000,16000))
    t18 = threading.Thread(target=generate_random_imgs, args=(1000,17000))
    t19 = threading.Thread(target=generate_random_imgs, args=(1000,18000))
    t20 = threading.Thread(target=generate_random_imgs, args=(1000,19000))
    # starting threads
    t1.start()
    t2.start()
    t3.start()
    t4.start()
    t5.start()
    t6.start()
    t7.start()
    t8.start()
    t9.start()
    t10.start()
    t11.start()
    t12.start()
    t13.start()
    t14.start()
    t15.start()
    t16.start()
    t17.start()
    t18.start()
    t19.start()
    t20.start()
  
    # wait until threads are completely executed
    t1.join()
    t2.join()
    t3.join()
    t4.join()
    t5.join()
    t6.join()
    t7.join()
    t8.join()
    t9.join()
    t10.join()
    t11.join()
    t12.join()
    t13.join()
    t14.join()
    t15.join()
    t16.join()
    t17.join()
    t18.join()
    t19.join()
    t20.join()
  
    # all threads completely executed
    print("Done!")

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

 Yeahhhhhh bitch! Done!


In [97]:
def main():
   
    folder = "equipped_c"
    for count, filename in enumerate(os.listdir(folder)):
        dst = f"equipped_c_{str(count)}.png"
        src =f"{folder}/{filename}"  # foldername/filename, if .py file is outside folder
        dst =f"{folder}/{dst}"
         
        # rename() function will
        # rename all the files
        os.rename(src, dst)
 

In [98]:
main()

In [83]:
background_file = path.join("backgrounds", "background1.png")
background_image = Image.open(background_file)
background_image = background_image.resize((1920,1080))

#Create character
police_character_file = path.join("characters/police_chars", "equipped_p/equipped_p_1.png")
police_character_image = Image.open(police_character_file)
police_character_image = police_character_image.resize((int(police_character_image.size[0]/3),int(police_character_image.size[1]/3)))
police_character_coordinates = (int(1920/2-police_character_image.width*1.6), int(1000-police_character_image.height)) #x, y
background_image.paste(police_character_image, police_character_coordinates, mask=police_character_image)

In [8]:
df = pd.DataFrame(columns = ["image", "civilian_present?", "civilian_armed?", "police_present?","police_armed?", "action"])
for root, dirs, files in os.walk(".", topdown=False):
    for name in files:
        if ".csv" in name:
            df_new = pd.read_csv(name)
            #print(df_new)
            df = df.append(df_new)
df.to_csv("data.csv")

In [18]:
#generate_all_imgs()
for i in range(1,number_of_threads+1):
    globals()['t%s'% str(i)] = threading.Thread(target=print_variable, args=(i,1000,(i-1)*1000))
# starting threads
for i in range(1,number_of_threads+1):
    globals()['t%s'% str(i)].start()
# wait until threads are completely executed
for i in range(1,number_of_threads+1):
    globals()['t%s'% str(i)].join()
print("All threads suucessfully executed")
# starting threads

499.5 0
499.5 1000
499.5 2000
499.5 3000
499.5 4000
499.5 5000
499.5 6000
499.5 7000
499.5 8000
499.5 9000
499.5 10000
499.5 11000
499.5 12000
499.5 13000
499.5 14000
499.5 15000
499.5 16000
499.5 17000
499.5 18000
499.5 19000
All threads suucessfully executed
